In [71]:
import pandas as pd
import cv2
import json
import matplotlib.pyplot as plt
from PIL import Image

from monai.transforms import (
    Compose,
    LoadImage,
    SpatialCrop,
    Resize,
    RandRotate,
    RandFlip,
    RandZoom,
    EnsureType,
    Lambda
)
from monai.data import Dataset, DataLoader

In [72]:
train_df = pd.read_excel(r'train\balanced_dataset.xlsx', index_col='Unnamed: 0')
train_df = train_df.sort_values('Unnamed: 0')
print(f'Количество исследований в сбалансированном датасете: {len(train_df)}')
train_df.head()

Количество исследований в сбалансированном датасете: 384


,ID,image_path,annotation_path_label,pathology
Unnamed: 0,,,,
0,image_393.jpg,train\images\image_393.jpg,train\annotation\annotation_393.json,1
2,image_290.jpg,train\images\image_290.jpg,train\annotation\annotation_290.json,1
3,image_206.jpg,train\images\image_206.jpg,train\annotation\annotation_206.json,1
4,image_429.jpg,train\images\image_429.jpg,train\annotation\annotation_429.json,0
4,image_429.jpg,train\images\image_429.jpg,train\annotation\annotation_429.json,0


In [73]:
# Вычисление средней высоты и ширины ROI
img_sizes = []
for img in train_df['image_path']:
    im = Image.open(img)
    width, height = im.size
    img_sizes.append([width, height])

print(f'Средняя ширина: {sum(img_sizes[0]) / len(img_sizes[0])}\n',
      f'Средняя высота: {sum(img_sizes[1]) / len(img_sizes[1])}')

Средняя ширина: 1415.0
 Средняя высота: 1801.0


In [74]:
# Загрузка изображения и JSON
image_path = train_df['image_path'][383]
json_path = train_df['annotation_path_label'][383]

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Конвертация в RGB для корректного отображения
with open(json_path, 'r') as f:
    data = json.load(f)

# Извлечение координат bbox
bbox = data['annotations'][0]['bbox']
x, y, w, h = map(int, bbox)

# Рисование прямоугольника
cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 10)

# Отображение с помощью matplotlib
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.axis('off')
plt.show()

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object


In [30]:
# Вычисление средней высоты и ширины
def load_bbox_wh(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    x, y, w, h = data['annotations'][0]['bbox']
    return w, h

In [31]:
load_bbox_wh(json_path = train_df['annotation_path_label'][383])

(138.60000000000002, 130.89999999999986)

In [65]:
# Функция для загрузки bbox из JSON файла
def load_bbox(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    bbox = data['annotations'][0]['bbox']
    return [int(x) for x in bbox]

# Функция для обрезки изображения по bbox
def crop_to_bbox(img_data):
    image, bbox = img_data['image'], img_data['bbox']
    y, x, h, w = bbox
    return {'image': image[:, y:y+h, x:x+w], 'label': img_data['label']}

In [67]:
load_bbox(json_path=train_df['annotation_path_label'][2])

[476, 1476, 91, 64]

# Как передать ROI в transform и нужно ли?

In [70]:
# Создание списка трансформаций, включая обрезку по bbox    
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        # SpatialCrop(roi_key="bbox"),  # Обрезка по bbox
        Resize((128, 128)),  # Изменение размера после обрезки
        # RandRotate(range_x=15, prob=0.5), # C 50% вероятностью поворачивает изображение на 15 градусов
        # RandFlip(spatial_axis=0, prob=0.5), # С 50% вероятностью отражает изображение по горизонтали
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5), # С вероятностью 50% зумит изображение от 0.9-1.1
        EnsureType()
    ]
)


OverflowError: Python int too large to convert to C long

In [ ]:
# Модифицированная функция для создания датасета MONAI
def create_dataset(dataframe):
    data = []
    for _, row in dataframe.iterrows():
        bbox = load_bbox(row['json_path'])  # Предполагается, что у вас есть столбец 'json_path' с путями к JSON файлам
        x, y, w, h = bbox
        data.append({
            "image": row['image_path'],
            "label": row['class'],
            "bbox": (y, x, y + h, x + w)  # MONAI использует формат (start_h, start_w, end_h, end_w)
        })
    return Dataset(data=data, transform=train_transforms)